# Scenario I: Selecting Best Hyperparameter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Seed for reproducibility
# np.random.seed(42)

# Generate synthetic data with multiple features
train_num = 100
n_features = 50

n_samples = 10000
X = np.random.rand(n_samples, n_features) * 10
true_coefs = np.ones(n_features)
y = X @ true_coefs + 2 + np.random.randn(n_samples) * 2

# Introduce some non-linearity to misspecify the model
y = y + 1 * np.sin(X).sum(axis=1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_num, random_state=42)

# Define a range of regularization parameters
alpha_values = np.linspace(0, 200, 100)


best_alpha_naive, cost_naive = 0, 0
best_alpha_cv5, cost_cv5 = 0, 0
best_alpha_loo, cost_loo = 0, 0
best_alpha_oracle = 0
loop_num = 100

for i in range(loop_num):
    # Naive training loss
    def naive_training_loss(X_train, y_train, alpha_values):
        losses = []
        for alpha in alpha_values:
            model = Ridge(alpha=alpha)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_train)
            loss = mean_squared_error(y_train, y_pred)
            losses.append(loss)
        return losses

    # 5-fold cross-validation
    def cross_validation_5fold(X_train, y_train, alpha_values):
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        avg_losses = []
        for alpha in alpha_values:
            fold_losses = []
            for train_index, val_index in kf.split(X_train):
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
                model = Ridge(alpha=alpha)
                model.fit(X_train_fold, y_train_fold)
                y_pred = model.predict(X_val_fold)
                loss = mean_squared_error(y_val_fold, y_pred)
                fold_losses.append(loss)
            avg_losses.append(np.mean(fold_losses))
        return avg_losses

    # Leave-one-out cross-validation
    def loo_cross_validation(X_train, y_train, alpha_values):
        loo = LeaveOneOut()
        avg_losses = []
        for alpha in alpha_values:
            fold_losses = []
            for train_index, val_index in loo.split(X_train):
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
                model = Ridge(alpha=alpha)
                model.fit(X_train_fold, y_train_fold)
                y_pred = model.predict(X_val_fold)
                loss = mean_squared_error(y_val_fold, y_pred)
                fold_losses.append(loss)
            avg_losses.append(np.mean(fold_losses))
        return avg_losses

    # Oracle test error
    def oracle_test_error(X_train, y_train, X_test, y_test, alpha_values):
        test_errors = []
        for alpha in alpha_values:
            model = Ridge(alpha=alpha)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            test_error = mean_squared_error(y_test, y_pred)
            test_errors.append(test_error)
        return test_errors

    # Calculate losses for each method
    naive_losses = naive_training_loss(X_train, y_train, alpha_values)
    cv5_losses = cross_validation_5fold(X_train, y_train, alpha_values)
    loo_losses = loo_cross_validation(X_train, y_train, alpha_values)
    oracle_errors = oracle_test_error(X_train, y_train, X_test, y_test, alpha_values)

# Find the best alpha for each method
    best_alpha_naive += alpha_values[np.argmin(naive_losses)] / loop_num
    best_alpha_cv5 += alpha_values[np.argmin(cv5_losses)] / loop_num
    best_alpha_loo += alpha_values[np.argmin(loo_losses)] / loop_num
    best_alpha_oracle += alpha_values[np.argmin(oracle_errors)] / loop_num

    cost_naive += oracle_errors[np.argmin(naive_losses)] / loop_num 
    cost_cv5 += oracle_errors[np.argmin(cv5_losses)] / loop_num
    cost_loo += oracle_errors[np.argmin(loo_losses)] / loop_num



print(f"Best alpha and cost (Naive Training Loss): {best_alpha_naive}, {cost_naive}")
print(f"Best alpha and cost (5-Fold Cross-Validation): {best_alpha_cv5}, {cost_naive}")
print(f"Best alpha and cost (Leave-One-Out Cross-Validation): {best_alpha_loo}, {cost_naive}")


# Scenario II: Selecting Different Model Class

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import random

loop_num = 100
# Generate synthetic data with multiple features
train_num = 400
n_samples = 10000
n_features = 10

naive_prob, naive_cost = 0, 0
cv_prob, cv_cost = 0, 0
loo_prob, loo_cost = 0, 0

for i in range(loop_num):
    print(i)
    X = np.random.rand(n_samples, n_features) * 10
    true_coefs = np.ones(n_features)
    y = X @ true_coefs + 2 + np.random.randn(n_samples) * 2

    # Introduce some non-linearity to misspecify the model
    alpha = random.uniform(0, 5)
    y = y + alpha * np.sin(X).sum(axis=1)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_num, random_state=42)

    # Define the models with fixed configurations
    models = {
        "Ridge": Ridge(alpha=1.0),  # Default alpha for Ridge
        "kNN": KNeighborsRegressor(n_neighbors = 5),  # Default n_neighbors for kNN
        "RandomForest": RandomForestRegressor(n_estimators = 50, max_samples = 0.5, random_state = 42)  # Default n_estimators for Random Forest
    }

    # Evaluate models using different criteria
    def evaluate_models(models, X_train, y_train, X_test, y_test):
        results = {}
        for model_name, model in models.items():
            # Naive training loss
            model.fit(X_train, y_train)
            y_pred_train = model.predict(X_train)
            naive_loss = mean_squared_error(y_train, y_pred_train)
            
            # 5-fold cross-validation
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            cv5_losses = []
            for train_index, val_index in kf.split(X_train):
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
                model.fit(X_train_fold, y_train_fold)
                y_pred_val = model.predict(X_val_fold)
                loss = mean_squared_error(y_val_fold, y_pred_val)
                cv5_losses.append(loss)
            avg_cv5_loss = np.mean(cv5_losses)
            
            # Leave-one-out cross-validation
            loo = LeaveOneOut()
            loo_losses = []
            for train_index, val_index in loo.split(X_train):
                X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
                y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
                model.fit(X_train_fold, y_train_fold)
                y_pred_val = model.predict(X_val_fold)
                loss = mean_squared_error(y_val_fold, y_pred_val)
                loo_losses.append(loss)
            avg_loo_loss = np.mean(loo_losses)
            
            # Oracle test error
            y_pred_test = model.predict(X_test)
            oracle_error = mean_squared_error(y_test, y_pred_test)
            
            # Store results
            results[model_name] = {
                "Naive Loss": naive_loss,
                "CV5 Loss": avg_cv5_loss,
                "LOO Loss": avg_loo_loss,
                "Oracle Error": oracle_error
            }
            
        return results

    # Evaluate all models
    results = evaluate_models(models, X_train, y_train, X_test, y_test)

    # Display results
    # for model_name, metrics in results.items():
    #     print(f"Model: {model_name}")
    #     for metric_name, value in metrics.items():
    #         print(f"  {metric_name}: {value:.4f}")

    # find avg cost and matches probability for each method

    ranked_items = sorted(results.items(), key=lambda item: item[1]["Oracle Error"])
    # Best performance index is ranked_keys[0]
    ranked_keys = [item[0] for item in ranked_items]

    rank_naive = sorted(results.items(), key=lambda item: item[1]["Naive Loss"])
    # Best performance
    ranked_naive_keys = [item[0] for item in rank_naive]
    if ranked_naive_keys[0] == ranked_keys[0]:
        naive_prob += 1
    naive_cost += results[ranked_naive_keys[0]]['Oracle Error'] / loop_num

    rank_cv = sorted(results.items(), key=lambda item: item[1]["CV5 Loss"])
    # Best performance
    ranked_cv_keys = [item[0] for item in rank_cv]
    if ranked_cv_keys[0] == ranked_keys[0]:
        cv_prob += 1
    cv_cost += results[ranked_cv_keys[0]]['Oracle Error'] / loop_num

    rank_loo = sorted(results.items(), key=lambda item: item[1]["LOO Loss"])
    # Best performance
    ranked_loo_keys = [item[0] for item in rank_loo]
    if ranked_loo_keys[0] == ranked_keys[0]:
        loo_prob += 1
    loo_cost += results[ranked_loo_keys[0]]['Oracle Error'] / loop_num
    
print('plugin', naive_prob / loop_num, naive_cost)
print('cv', cv_prob / loop_num, cv_cost)
print('loo', loo_prob  / loop_num, loo_cost)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
plugin 0.71 39.94704327392565
cv 0.96 37.25798923636545
loo 0.95 37.259071531396415
